In [1]:
import threading
from IPython.display import display
import numpy as np
from ipywidgets import widgets
from ipywidgets.widgets import Label, FloatProgress, FloatSlider
from ipywidgets.widgets import Layout, HBox, VBox
import bqplot as bq
from ipywidgets.widgets import Audio

from predict_syn import Predict_Syn

In [2]:
ps = Predict_Syn()
cmp_mean_vector, cmp_std_vector, test_lin_x, test_lab_x = ps.load_prev_fea()
v1_array, _ , f0, mgc, bap = ps.parms_gen(0, 0, 100, test_lin_x, test_lab_x, cmp_mean_vector, cmp_std_vector)
contents, intensity = ps.gen_wav(f0, mgc, bap)
color = ps.load_casia_color()

In [3]:
au = widgets.Audio(value=contents, format="wav",loop=False)
# Adding default tooltip to Line Chart
x_sc = bq.LinearScale()
y_sc = bq.LinearScale(min=20,)
x_ax = bq.Axis(label='time', scale=x_sc)
y_ax = bq.Axis(label='F0', scale=y_sc, orientation="vertical")
x_data = np.arange(len(f0.flatten()))
y_data = f0.flatten()
def_tt = bq.Tooltip(fields=['y'], formats=['.2f'])
line_chart = bq.Scatter(x=x_data, y=y_data, tooltip=def_tt,scales= {'x': x_sc, 'y': y_sc},default_size=5)

# Adding default tooltip to Line Chart
x_sc2 = bq.LinearScale()
y_sc2 = bq.LinearScale()
x_ax2 = bq.Axis(label='time', scale=x_sc2)
y_ax2 = bq.Axis(label='mgc', scale=y_sc2, orientation="vertical")
def_tt2 = bq.Tooltip(fields=['name', 'index'], formats=['', '.2f'])
x_data = np.arange(mgc.shape[0])
y_data = mgc[:,0:5]
line_chart_2 = bq.Lines(x=x_data, y=y_data.T, tooltip=def_tt2, scales= {'x': x_sc2, 'y': y_sc2}, labels=["mgc 1", "mgc 2", "mgc 3", "mgc 4", "mgc 5"],display_legend=True )

# Adding default tooltip to Line Chart
x_sc3 = bq.LinearScale()
y_sc3 = bq.LinearScale()
x_ax3 = bq.Axis(label='time', scale=x_sc3)
y_ax3 = bq.Axis(label='intensity', scale=y_sc3, orientation="vertical")
def_tt3 = bq.Tooltip(fields=['y'], formats=['.2f'])
y_data = intensity
x_data = np.arange(intensity.shape[0])
line_chart_3 = bq.Scatter(x=x_data, y=y_data, tooltip=def_tt3,scales= {'x': x_sc3, 'y': y_sc3},default_size=3)


fig1 = bq.Figure(layout=Layout(width="500px",height="300px"),
                axes=[x_ax, y_ax],
                marks=[line_chart])
fig2 = bq.Figure(layout=Layout(width="500px",height="300px"),
                axes=[x_ax2, y_ax2],
                marks=[line_chart_2])
fig3 = bq.Figure(layout=Layout(width="500px",height="300px"),
                axes=[x_ax3, y_ax3],
                marks=[line_chart_3])
wA1 = FloatSlider(
value=0,min=-2,max=2,step=0.01,description="control 1:",layout=Layout(width='490px',margin='0 0 5px 0')
)
wA2 = FloatSlider(
value=0,min=-2,max=2,step=0.01,description="control 2:",layout=Layout(width='490px',margin='0 0 5px 0')
)

# load color map
color_all = np.zeros(ps.training_num + 1)
color_all[0:ps.training_num]=color
color_all[-1]=6
x_sc4 = bq.LinearScale()
y_sc4 = bq.LinearScale()
col_line = bq.ColorScale(colors=['green', 'white','black','blue','magenta','yellow', 'red'])

x_ax4 = bq.Axis(label='1d', scale=x_sc4)
y_ax4 = bq.Axis(label='2d', scale=y_sc4, orientation="vertical")
def_tt4 = bq.Tooltip(fields=['x','y'], formats=['.2f','.2f'])
x_data = np.zeros(ps.training_num + 1)
y_data = np.zeros(ps.training_num + 1)
x_data[0:ps.training_num] = v1_array[:,0]
y_data[0:ps.training_num] = v1_array[:,1]
x_data[-1] = 1.5
y_data[-1] = 1.5

line_chart_4 = bq.Scatter(x=x_data, y=y_data, tooltip=def_tt4,scales= {'x': x_sc4, 'y': y_sc4,'color':col_line},color=color_all, default_size=3)
fig4 = bq.Figure(layout=Layout(width="500px",height="500px"),
                axes=[x_ax4, y_ax4],
                marks=[line_chart_4])
hb1 = HBox(children=(fig1,fig2))
hb2 = HBox(children=(fig3,fig4))
hb3 = HBox(children=(VBox(children=(wA1,wA2)),au))

box = VBox(children=(hb1, hb2,hb3), layout=Layout(border="solid spx gray",width="900px"))

In [4]:
def work():
    old_w1 = 0
    old_w2 = 0
    while True:
        if old_w1 != wA1.value or old_w2 != wA2.value:
            _, _ , f0, mgc, bap = ps.parms_gen( wA1.value, wA2.value, 100, test_lin_x, test_lab_x, cmp_mean_vector, cmp_std_vector)
            contents, intensity = ps.gen_wav(f0, mgc, bap)
            
            line_chart.x = np.arange(len(f0.flatten()))
            line_chart.y = f0.flatten()
            line_chart_2.y = mgc[:,0:5].T
            line_chart_2.x = np.arange(mgc.shape[0])
  
            line_chart_3.y = intensity
            line_chart_3.x = np.arange(intensity.shape[0])
            line_chart_4.x = np.append(v1_array[:,0],wA1.value)
            line_chart_4.y = np.append(v1_array[:,1],wA2.value)
            
            au.value = contents
            old_w1 = wA1.value
            old_w2 = wA2.value
        

In [5]:
thread = threading.Thread(target=work)
display(box)
thread.start()